# 300_Join_FIFA_Fantasy1718 
## Purpose 
In this notebook we read both our Fantasty and FIFA cleaned datasets and join them and performing any necessary cleaning.  
## Datasets 
* _Input_: FIFA18.csv,Fantasy1718.csv
* _Output_: Joined1718.csv

In [1]:
import os.path
import numpy as np
import pandas as pd

### Reading in our cleaned FIFA and Fantasy datasets.

In [2]:
Fifa = pd.read_csv("../../data/prep/FIFA18.csv")
Fantasy1718 = pd.read_csv("../../data/prep/Fantasy1718.csv")

### Merging the two datasets on Surname and First Initial
* Use pd.merge to join the tables  .
* Drop irrelevant column 'Unnamed: 0_x'.

In [3]:
Join1718 = pd.merge(Fifa, Fantasy1718, on=['Surname','FirstInitial'])
Join1718.drop(Join1718.columns[[0]], axis=1, inplace=True)

In [4]:
Join1718.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,5,0,1,0,0,0,25,7.280000,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,3,0,0,2,0,0,0,32,7.375000,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,0,0,0,0,0,0,0,20,8.550000,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,0,2,0,0,0,0,0,28,6.535714,Luis
4,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,1,0,0,0,0,0,0,1,1.000000,Leonardo


### Creating a while loop to remove players
* Removes all players thats Firstname is not contained in 'name'
* Removes all players thats Firstname is not contained in 'full_name'.

In [5]:
i = 0
array = []

while i < len(Join1718):
    if Join1718['Firstname'][i] in Join1718['name'][i] or Join1718['Firstname'][i] in Join1718['full_name'][i]:
        array.append(True)
        i+=1
    else:
        array.append(False)
        i+=1

### Making the array 'array' a numpy array 
* Making the numpy array into a dataframe.

In [6]:
array = np.array(array)
Join1718 = Join1718[array]

### Creating a dataframe ERROR with all players that have problems
* Contains players in Join that are duplicated using 'Players' or 'full_name'.

In [7]:
Error1718 = Join1718[(Join1718['Players'].isin(Join1718['Players'][Join1718['Players'].duplicated()]) | Join1718['full_name'].isin(Join1718['full_name'][Join1718['full_name'].duplicated()]))]
Error1718.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
28,188567,P. Aubameyang,Pierre-Emerick Aubameyang,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2073,28,German Bundesliga,1989-06-18,187.0,...,1,1,0,1,0,0,0,9,6.000000,Pierre
29,188567,P. Aubameyang,Pierre-Emerick Aubameyang,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2073,28,German Bundesliga,1989-06-18,187.0,...,0,1,0,1,0,0,0,16,6.437500,Pierre
49,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,2,4,0,0,31,12,0,25,3.120000,Marcelo
50,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,0,1,0,0,34,15,1,33,3.575758,Marcelo
51,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,2,4,0,0,31,12,0,25,3.120000,Marcelo


### Creating a dataframe TopLeagues with all players inside the Top 5 leagues
* Takes all players in ERROR insode the Top 5 leagues using str.contains.

In [8]:
leagues = ['German Bundesliga','Spanish Primera Division','English Premier League','French Ligue 1','Italian Serie A']
TopLeagues1718 = Error1718[(Error1718.league.str.contains('|'.join(leagues)) == True) | Error1718['league'].isnull()]

### Creating a dataframe DELETE with all players outside the Top 5 leagues
* Takes all players in ERROR outside the Top 5 leagues using str.contains.

In [9]:
Delete1718 = Error1718[Error1718.league.str.contains('|'.join(leagues)) == False]

### Creating a dataframe FIX
* This dataframe takes all players in the Top 5 that have been duplicated .

In [10]:
Fix1718 = TopLeagues1718[(TopLeagues1718['Players'].isin(TopLeagues1718['Players'][TopLeagues1718['Players'].duplicated()]) | TopLeagues1718['full_name'].isin(TopLeagues1718['full_name'][TopLeagues1718['full_name'].duplicated()]))]
Fix1718.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
28,188567,P. Aubameyang,Pierre-Emerick Aubameyang,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2073,28,German Bundesliga,1989-06-18,187.0,...,1,1,0,1,0,0,0,9,6.000000,Pierre
29,188567,P. Aubameyang,Pierre-Emerick Aubameyang,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2073,28,German Bundesliga,1989-06-18,187.0,...,0,1,0,1,0,0,0,16,6.437500,Pierre
49,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,2,4,0,0,31,12,0,25,3.120000,Marcelo
50,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,0,1,0,0,34,15,1,33,3.575758,Marcelo
51,180334,Marcelo,Marcelo Antonio Guedes Filho,Olympique Lyonnais,https://cdn.sofifa.org/18/teams/66.png,1789,30,French Ligue 1,1987-05-20,191.0,...,2,4,0,0,31,12,0,25,3.120000,Marcelo


### Removing players from JOIN
* Removing players whos full name is in Delete and have the same birth date.
* Removing players whos full name is in Fix and have the same birth date.

In [11]:
Join1718 = Join1718[(~(Join1718.full_name.isin(Delete1718.full_name) & Join1718.birth_date.isin(Delete1718.birth_date)))]
Join1718 = Join1718[(~(Join1718.full_name.isin(Fix1718.full_name) & Join1718.birth_date.isin(Fix1718.birth_date)))]

### Removing players from FIX
* Removing players thats 'full_name' is not equal to 'Players'.
* Dropping all duplicate players in Fixed.

In [12]:
Fixed1718 = Fix1718[Fix1718['full_name'] == Fix1718['Players']]
Fixed1718 = Fixed1718[(Fixed1718['Players'].isin(Fixed1718['Players'][Fixed1718['Players'].duplicated()]) | Fixed1718['full_name'].isin(Fixed1718['full_name'][Fixed1718['full_name'].duplicated()]))]
Fixed1718 = Fixed1718[~Fixed1718.duplicated(['Players', 'Clubs', 'Points'])]
Fixed1718.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
144,170890,B. Matuidi,Blaise Matuidi,Juventus,https://cdn.sofifa.org/18/teams/45.png,2172,30,Italian Serie A,1987-04-09,175.0,...,5,5,0,0,0,0,0,30,2.333333,Blaise
145,170890,B. Matuidi,Blaise Matuidi,Juventus,https://cdn.sofifa.org/18/teams/45.png,2172,30,Italian Serie A,1987-04-09,175.0,...,2,0,0,0,0,0,0,2,1.000000,Blaise
155,212218,A. Laporte,Aymeric Laporte,Athletic Club de Bilbao,https://cdn.sofifa.org/18/teams/448.png,1908,23,Spanish Primera Division,1994-05-27,189.0,...,0,1,0,0,4,0,0,7,3.571429,Aymeric
156,212218,A. Laporte,Aymeric Laporte,Athletic Club de Bilbao,https://cdn.sofifa.org/18/teams/448.png,1908,23,Spanish Primera Division,1994-05-27,189.0,...,0,0,0,0,22,10,1,19,2.684211,Aymeric
240,179784,B. Howedes,Benedikt Howedes,Juventus,https://cdn.sofifa.org/18/teams/45.png,1862,29,Italian Serie A,1988-02-29,187.0,...,0,0,0,0,0,0,0,0,0.000000,Benedikt


### Adding players who have transfered teams
* Get the sum of all relevant columns to get one full player

In [13]:
d = {
 'ID' : 'first',
 'club_logo':'first',
 'special':'first',
 'age':'first',
 'league':'first',
 'birth_date':'first',
 'height_cm':'first',
 'weight_kg':'first',
 'body_type':'first',
 'real_face':'first',
 'flag':'first',
 'nationality':'first',
 'photo':'first',
 'eur_value':'first',
 'eur_wage':'first',
 'eur_release_clause':'first',
 'overall':'first',
 'potential':'first',
 'pac':'first',
 'sho':'first',
 'pas':'first',
 'dri':'first',
 'def':'first',
 'phy':'first',
 'international_reputation':'first',
 'skill_moves':'first',
 'weak_foot':'first',
 'work_rate_att':'first',
 'work_rate_def':'first',
 'preferred_foot':'first',
 'crossing':'first',
 'finishing':'first',
 'heading_accuracy':'first',
 'short_passing':'first',
 'volleys':'first',
 'dribbling':'first',
 'curve':'first',
 'free_kick_accuracy':'first',
 'long_passing':'first',
 'ball_control':'first',
 'acceleration':'first',
 'sprint_speed':'first',
 'agility':'first',
 'reactions':'first',
 'balance':'first',
 'shot_power':'first',
 'jumping':'first',
 'stamina':'first',
 'strength':'first',
 'long_shots':'first',
 'aggression':'first',
 'interceptions':'first',
 'positioning':'first',
 'vision':'first',
 'penalties':'first',
 'composure':'first',
 'marking':'first',
 'standing_tackle':'first',
 'sliding_tackle':'first',
 'gk_diving':'first',
 'gk_handling':'first',
 'gk_kicking':'first',
 'gk_positioning':'first',
 'gk_reflexes':'first',
 'rs':'first',
 'rw':'first',
 'rf':'first',
 'ram':'first',
 'rcm':'first',
 'rm':'first',
 'rdm':'first',
 'rcb':'first',
 'rb':'first',
 'rwb':'first',
 'st':'first',
 'lw':'first',
 'cf':'first',
 'cam':'first',
 'cm':'first',
 'lm':'first',
 'cdm':'first',
 'cb':'first',
 'lb':'first',
 'lwb':'first',
 'ls':'first',
 'lf':'first',
 'lam':'first',
 'lcm':'first',
 'ldm':'first',
 'lcb':'first',
 'gk':'first',
 '1_on_1_rush_trait':'first',
 'acrobatic_clearance_trait':'first',
 'argues_with_officials_trait':'first',
 'avoids_using_weaker_foot_trait':'first',
 'backs_into_player_trait':'first',
 'bicycle_kicks_trait':'first',
 'cautious_with_crosses_trait':'first',
 'chip_shot_trait':'first',
 'chipped_penalty_trait':'first',
 'comes_for_crosses_trait':'first',
 'corner_specialist_trait':'first',
 'diver_trait':'first',
 'dives_into_tackles_trait':'first',
 'diving_header_trait':'first',
 'driven_pass_trait':'first',
 'early_crosser_trait':'first',
 "fan's_favourite_trait":'first',
 'fancy_flicks_trait':'first',
 'finesse_shot_trait':'first',
 'flair_trait':'first',
 'flair_passes_trait':'first',
 'gk_flat_kick_trait':'first',
 'gk_long_throw_trait':'first',
 'gk_up_for_corners_trait':'first',
 'giant_throw_in_trait':'first',
 'inflexible_trait':'first',
 'injury_free_trait':'first',
 'injury_prone_trait':'first',
 'leadership_trait':'first',
 'long_passer_trait':'first',
 'long_shot_taker_trait':'first',
 'long_throw_in_trait':'first',
 'one_club_player_trait':'first',
 'outside_foot_shot_trait':'first',
 'playmaker_trait':'first',
 'power_free_kick_trait':'first',
 'power_header_trait':'first',
 'puncher_trait':'first',
 'rushes_out_of_goal_trait':'first',
 'saves_with_feet_trait':'first',
 'second_wind_trait':'first',
 'selfish_trait':'first',
 'skilled_dribbling_trait':'first',
 'stutter_penalty_trait':'first',
 'swerve_pass_trait':'first',
 'takes_finesse_free_kicks_trait':'first',
 'target_forward_trait':'first',
 'team_player_trait':'first',
 'technical_dribbler_trait':'first',
 'tries_to_beat_defensive_line_trait':'first',
 'poacher_speciality':'first',
 'speedster_speciality':'first',
 'aerial_threat_speciality':'first',
 'dribbler_speciality':'first',
 'playmaker_speciality':'first',
 'engine_speciality':'first',
 'distance_shooter_speciality':'first',
 'crosser_speciality':'first',
 'free_kick_specialist_speciality':'first',
 'tackling_speciality':'first',
 'tactician_speciality':'first',
 'acrobat_speciality':'first',
 'strength_speciality':'first',
 'clinical_finisher_speciality':'first',
 'prefers_rs':'first',
 'prefers_rw':'first',
 'prefers_rf':'first',
 'prefers_ram':'first',
 'prefers_rcm':'first',
 'prefers_rm':'first',
 'prefers_rdm':'first',
 'prefers_rcb':'first',
 'prefers_rb':'first',
 'prefers_rwb':'first',
 'prefers_st':'first',
 'prefers_lw':'first',
 'prefers_cf':'first',
 'prefers_cam':'first',
 'prefers_cm':'first',
 'prefers_lm':'first',
 'prefers_cdm':'first',
 'prefers_cb':'first',
 'prefers_lb':'first',
 'prefers_lwb':'first',
 'prefers_ls':'first',
 'prefers_lf':'first',
 'prefers_lam':'first',
 'prefers_lcm':'first',
 'prefers_ldm':'first',
 'prefers_lcb':'first',
 'prefers_gk':'first',
 'Players':'first',
 'Position':'first',
 'Clubs':'first',
 'Points':'sum',  
 'Start':'sum', 
 'Sub':'sum', 
 'club':'last',
 'Position':'first',
 'Start':'sum',
 'Goals':'sum',
 'A':'sum',
 'CS':'sum',
 'CS part':'sum',
 'Yellow':'sum',
 'Red':'sum',
 'Sub':'sum',
 'CO':'sum',
 'Off':'sum',
 'Pen SV':'sum',
 'Pen M':'sum',
 'Goals conceded':'sum',
 'Conceded 1+':'sum',
 'OG':'sum',
 'Firstname':'first',
 'Apps':'sum',
 'Form': 'mean'
}

#### Groupby function applying 'd' to fix joining the players.

In [14]:
Fixed1718 = (Fixed1718.groupby(['name','full_name'], sort=False, as_index=False).agg(d).reindex(columns=Fix1718.columns))
Fixed1718.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,170890,B. Matuidi,Blaise Matuidi,Juventus,https://cdn.sofifa.org/18/teams/45.png,2172,30,Italian Serie A,1987-04-09,175.0,...,7,5,0,0,0,0,0,32,1.666667,Blaise
1,212218,A. Laporte,Aymeric Laporte,Athletic Club de Bilbao,https://cdn.sofifa.org/18/teams/448.png,1908,23,Spanish Primera Division,1994-05-27,189.0,...,0,1,0,0,26,10,1,26,3.127820,Aymeric
2,179784,B. Howedes,Benedikt Howedes,Juventus,https://cdn.sofifa.org/18/teams/45.png,1862,29,Italian Serie A,1988-02-29,187.0,...,0,1,0,0,1,0,0,3,2.666667,Benedikt
3,167431,G. Castro,Gonzalo Castro,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2156,30,German Bundesliga,1987-06-11,172.0,...,3,7,0,0,0,0,0,19,2.000000,Gonzalo
4,181820,S. Jovetic,Stevan Jovetic,AS Monaco,https://cdn.sofifa.org/18/teams/69.png,1977,27,French Ligue 1,1989-11-02,183.0,...,8,5,0,0,0,0,0,15,2.133333,Stevan


#### Quick look at what FIXED with most relevant columns

In [15]:
Fixed1718[['full_name','club','Players','Clubs','Points']].head(10)

,full_name,club,Players,Clubs,Points
0,Blaise Matuidi,Juventus,Blaise Matuidi,JUVE,72
1,Aymeric Laporte,Athletic Club de Bilbao,Aymeric Laporte,MCFC,76
2,Benedikt Howedes,Juventus,Benedikt Howedes,SCH4,16
3,Gonzalo Castro,Borussia Dortmund,Gonzalo Castro,BD1,38
4,Stevan Jovetic,AS Monaco,Stevan Jovetic,INTE,64
5,Michy Batshuayi,Chelsea,Michy Batshuayi,CHEL,80
6,Hector Moreno,Roma,Hector Moreno,RESO,46
7,Davide Zappacosta,Chelsea,Davide Zappacosta,CHEL,43
8,Paul-Georges Ntep,VfL Wolfsburg,Paul-Georges Ntep,VFL,25
9,Francis Coquelin,Arsenal,Francis Coquelin,ARSL,27


### Joining Join and Fixed
* Using pd.concat to join the two dataframes together.
* Drop irrelevant column.

In [16]:
Joined1718 = pd.concat([Join1718,Fixed1718])
Joined1718 = Joined1718.reset_index()
Joined1718.drop(Joined1718.columns[[0]], axis=1, inplace=True)
Joined1718.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,5,0,1,0,0,0,25,7.280000,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,3,0,0,2,0,0,0,32,7.375000,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,0,0,0,0,0,0,0,20,8.550000,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,0,2,0,0,0,0,0,28,6.535714,Luis
4,225591,L. Suarez,Leonardo Suarez,Villarreal CF,https://cdn.sofifa.org/18/teams/483.png,1673,21,Spanish Primera Division,1996-03-30,167.0,...,1,0,0,0,0,0,0,1,1.000000,Leonardo


#### Saving to csv file in data/prep

In [17]:
Joined1718.to_csv('../../data/prep/Joined1718.csv')